In [1]:
import sys
sys.path.append("..")
import numpy as np

from layers import *

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# test conv1x1 gathered scheduler
# conv1x1_gathered_scheduler = Conv1x1GatheredScheduler(512, 4, 128, 2, "log/conv3x3_gathered.log")
# conv1x1_gathered_scheduler.n_trial = 50
# conv1x1_gathered_scheduler.rtol = 1e-5
# conv1x1_gathered_scheduler.atol = 1e-3
# conv1x1_gathered_scheduler.autotune(refresh=True)
# conv1x1_gathered_scheduler.build(display=False)
# conv1x1_gathered_scheduler.check(runtype="pytorch")
# conv1x1_gathered_scheduler.evaluate()

/usr/local/lib/python3.8/dist-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


0.00025236856

In [2]:
# test scatter add
# scatter_add_scheduler = ScatterAddScheduler(64, 512, 128, 2, "log/scatter_add.log")
# scatter_add_scheduler.n_trial = 10
# scatter_add_scheduler.autotune(refresh=True)
# scatter_add_scheduler.build(display=False)
# scatter_add_scheduler.check()
# scatter_add_scheduler.evaluate()

0.00018825887000000003

In [ ]:
# test conv3x3 gathered scheduler
# conv3x3_gathered_scheduler = Conv3x3GatheredScheduler(512, 4, 128, 2, "log/conv3x3_gathered.log")
# conv3x3_gathered_scheduler.n_trial = 50
# conv3x3_gathered_scheduler.rtol = 1e-5
# conv3x3_gathered_scheduler.atol = 1e-3
# conv3x3_gathered_scheduler.autotune(refresh=True)
# conv3x3_gathered_scheduler.build(display=False)
# conv3x3_gathered_scheduler.check(runtype="pytorch")

# sample = conv3x3_gathered_scheduler._generate_sample()
# res_tvm = conv3x3_gathered_scheduler._run_tvm(sample)
# res_pytorch = conv3x3_gathered_scheduler._run_pytorch(sample)
# for i in range(128):
#     for j in range(512):
#         if np.sum(res_tvm[i, j, :] - res_pytorch[i, j, :]) > 1e-2:
#             print(i, j)

# conv3x3_gathered_scheduler.evaluate()

0.0005954454

In [ ]:
# test gather scheduler
# gather_scheduler = GatherScheduler(64, 512, 128, 2, "log/gather.log")
# gather_scheduler.n_trial = 10
# gather_scheduler.autotune(refresh=True)
# gather_scheduler.build(display=False)
# gather_scheduler.check()
# gather_scheduler.evaluate()

/usr/local/lib/python3.8/dist-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


2.398621e-05